In [1]:
import os

import pandas

import dask.dataframe as dd

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

from tqdm.auto import tqdm
tqdm.pandas(desc="global")

In [3]:
from tqdm import tqdm

class InnerProgressBar(tqdm):
	def __init__(self, total, desc):
		super().__init__(desc=desc)
		self.total = total
		self.current = 0

	def update_to(self):
		self.update(self.current)

In [4]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_raw_data_path = data_path + "raw_data.parquet"

parquet_process_data_path = data_path + "processed_data.parquet"

In [5]:
os.makedirs(data_path, exist_ok=True)

print(f"Data Path:\t{data_path}\n\nExists:\t{os.path.exists(data_path)}")

Data Path:	/data/parquet/

Exists:	True


In [6]:
pandas_df = None
if os.path.exists("foo"):
	print("Loading from parquet")
	pandas_df = pandas.read_parquet(parquet_raw_data_path)


else:
	print("Loading from table")
	raw_data = table_adapter.get_all_entities(table_name)
	pandas_df = pandas.DataFrame(list(raw_data))
	print(f"Saving to parquet {parquet_process_data_path}")
	pandas_df.to_parquet(parquet_raw_data_path)

Loading from table
Saving to parquet /data/parquet/processed_data.parquet


In [7]:
display(f"Initial Dataframe Shape:\t{pandas_df.shape}")

display(pandas_df)

'Initial Dataframe Shape:\t(16085, 19)'

,PartitionKey,RowKey,author,caption,curated,exists,exits,flair,hash,id,image,image_name,permalink,small_image,subreddit,text,updated_caption,updated_path,url
0,training,1000cej,OtterlyFoxy,cars are parked on the side of the road in the...,False,True,True,NaN,7a8d96e378c15c8ab8440ac311f12c11,1000cej,D:\data\images\CityPorn\4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,CityPorn,New York in the fog,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",D:\data\images\CityPorn\4emw5uldib9a1.jpg,https://i.redd.it/4emw5uldib9a1.jpg
1,training,1000d16,princessxo699,a woman sitting on a table holding a pink flower,False,True,True,Outfit of the Day,9951b4f82caeb8ba2bd9f79f8d422450,1000d16,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,GgFEagO.jpg,/r/SFWNextDoorGirls/comments/1000d16/thoughts_...,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,SFWNextDoorGirls,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,https://i.imgur.com/GgFEagO.jpg
2,training,1000fg0,BlkBrd1312,a woman in a bikini with a cell phone in her hand,False,True,True,HotGirlNextDoor,be8dd55e34216bec1e15e03fa296eacc,1000fg0,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,nwa7hts2jb9a1.jpg,/r/HotGirlNextDoor/comments/1000fg0/iktr/,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,HotGirlNextDoor,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,https://i.redd.it/nwa7hts2jb9a1.jpg
3,training,1000glf,toolate_sharkbait,blonde woman with blonde hair and tattoos on h...,False,True,True,Female (18+),e554c1ed7ffa2740436ac082068b2824,1000glf,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,/r/AmIhotAF/comments/1000glf/just_looking_for_...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,AmIhotAF,Just looking for entertainment,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,https://i.redd.it/4xyb1vgbjb9a1.jpg
4,training,1000j1n,trent8051,a man with a beard and a beard sitting in fron...,False,True,NaN,NaN,1dec3dabb5e46cde01855d06089c287a,1000j1n,D:\data\images\greentext\3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,/r/greentext/comments/1000j1n/anon_wants_elon_...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,greentext,Anon wants Elon cut,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",NaN,https://i.redd.it/3mewbe0wjb9a1.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16080,training,zzzg13,AdConsistent3184,A little girl in a pink dress is sitting on a ...,False,False,False,NaN,ba89401054b09d3ff2b608705f8a7110,zzzg13,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,3rv0ql34bb9a1.jpg,/r/SFWRedheads/comments/zzzg13/cute_but_deadly/,,SFWRedheads,Cute but Deadly 😋,NaN,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,https://i.redd.it/3rv0ql34bb9a1.jpg
16081,training,zzzlbf,Slipstream_Valet,a man in a red dress and a woman in a red dress,False,True,NaN,NaN,5007b937974ae333022c0c91b795ca09,zzzlbf,/data/images/trippinthroughtime/arCpzQ0.jpg,arCpzQ0.jpg,/r/trippinthroughtime/comments/zzzlbf/he_didnt...,/data/images/trippinthroughtime/thumbnail/arCp...,trippinthroughtime,He didn't shed light on the topic I guess.,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",NaN,https://i.imgur.com/arCpzQ0.jpg
16082,training,zzzlyl,Particular-Chip2355,Error in captioning image,False,False,True,NaN,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,Minquan_Bridge2017_TAIWAN.jpg,/r/CityPorn/comments/zzzlyl/taipei_taiwan/,,CityPorn,"Taipei, Taiwan",NaN,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,https://upload.wikimedia.org/wikipedia/commons...
16083,training,zzzrv5,Mary_ford1,blond woman in black lingersuit sitting on a c...,False,True,NaN,NaN,fb0bf1ab59095edf938ece2e036dae86,zzzrv5,/data/images/RealGirls_SFW/2gxZ9rS.jpg,

In [8]:
print("Normalizing Dataframe For Processing")

subreddit = pandas_df["subreddit"]
image = pandas_df["image_name"]
text = pandas_df["text"]
hash_value = pandas_df["hash"]
original_caption = pandas_df["caption"]
thumbnail_caption = pandas_df["updated_caption"]
comment_id = pandas_df["id"]
all_normalized_frame = pandas.DataFrame(
	{
		"subreddit": subreddit,
		"file_name": image,
		"text": text,
		"original_caption": original_caption,
		"thumbnail_path": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}", axis=1),
		"thumbnail_caption": thumbnail_caption,
		"thumbnail_exists": pandas_df.apply(lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"original_image": pandas_df.apply(lambda x: f"D:\\data\\images\\{x['subreddit']}\\{x['image_name']}", axis=1),
		"original_image_exists": pandas_df.apply(lambda x: os.path.exists(f"D:\\data\\images\\{x['subreddit']}\\thumbnail\\{x['image_name']}"), axis=1),
		"hash": hash_value,
		"id": comment_id
	}
)
display(f"Normalized Dataframe Shape:\t{all_normalized_frame.shape}")
display(f"Saving to parquet {parquet_process_data_path}")
all_normalized_frame.to_parquet(parquet_process_data_path)

Normalizing Dataframe For Processing


'Normalized Dataframe Shape:\t(16085, 11)'

'Saving to parquet /data/parquet/processed_data.parquet'

In [9]:
display(f"Reading from parquet {parquet_process_data_path}")
all_normalized_frame = pandas.read_parquet(parquet_process_data_path)
display(all_normalized_frame)

'Reading from parquet /data/parquet/processed_data.parquet'

,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,cars are parked on the side of the road in the...,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
1,SFWNextDoorGirls,GgFEagO.jpg,Thoughts about my NYE party outfit?,a woman sitting on a table holding a pink flower,D:\data\images\SFWNextDoorGirls\thumbnail\GgFE...,a woman sitting on a table holding a pink flower,False,D:\data\images\SFWNextDoorGirls\GgFEagO.jpg,False,9951b4f82caeb8ba2bd9f79f8d422450,1000d16
2,HotGirlNextDoor,nwa7hts2jb9a1.jpg,(IKTR),a woman in a bikini with a cell phone in her hand,D:\data\images\HotGirlNextDoor\thumbnail\nwa7h...,a woman in a bikini with a cell phone in her hand,False,D:\data\images\HotGirlNextDoor\nwa7hts2jb9a1.jpg,False,be8dd55e34216bec1e15e03fa296eacc,1000fg0
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
...,...,...,...,...,...,...,...,...,...,...,...
16080,SFWRedheads,3rv0ql34bb9a1.jpg,Cute but Deadly 😋,A little girl in a pink dress is sitting on a ...,D:\data\images\SFWRedheads\thumbnail\3rv0ql34b...,NaN,False,D:\data\images\SFWRedheads\3rv0ql34bb9a1.jpg,False,ba89401054b09d3ff2b608705f8a7110,zzzg13
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red dress and a woman in a red dress,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf
16082,CityPorn,Minquan_Bridge2017_TAIWAN.jpg,"Taipei, Taiwan",Error in captioning image,D:\data\images\CityPorn\thumbnail\Minquan_Brid...,NaN,False,D:\data\images\CityPorn\Minquan_Bridge2017_TAI...,False,3cac9b29f27d5fc26ddcec4e1641649f,zzzlyl
16083,RealGirls_SFW,2gxZ9rS.jpg,My tender body needs caress,blond woman in black lingersuit sitting on a c...,D:\data\images\RealGirls_SFW\thumbnail\2gxZ9rS...,"""blond|NN"",""woman|NN"",""in|IN"",""black|JJ"",""ling...",False,D:\data\images\RealGirls_SFW\2gxZ9rS.jpg,False,fb0bf1ab59095edf938ece2e036dae86,zzzrv5


In [10]:
display("Filtering Subreddits with Images")
filtered_on_exist = all_normalized_frame[(all_normalized_frame["thumbnail_exists"] == True) & (all_normalized_frame["original_image_exists"] == True)]

total_images_display = filtered_on_exist.groupby("subreddit").size().reset_index(name="count")
display(total_images_display.sort_values("count", ascending=False))
display(f"Total Records {total_images_display['count'].sum()}")

'Filtering Subreddits with Images'

,subreddit,count
3,EarthPorn,1883
2,CityPorn,1795
13,memes,1180
16,spaceporn,1101
4,Faces,1028
9,SFWRedheads,984
8,SFWNextDoorGirls,848
12,greentext,713
17,trippinthroughtime,478
15,sfwpetite,288


'Total Records 11514'

In [11]:
display("Data With Known Images")
display(filtered_on_exist)

'Data With Known Images'

,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,cars are parked on the side of the road in the...,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,a view of a view of a large green and purple a...,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,a scene of a boat is sitting on the shore of a...,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,starrdust sky with a few stars and a few stars,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,a picture taken from the earth's surface of th...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5
16079,greentext,bgho6WK.jpg,Anon does a little trolling,a screenshote of a text message from a man who...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red dress and a woman in a red dress,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf


In [12]:
from shared_code.utility.scripts import blip_caption
blip_caption = blip_caption.BlipCaption(1)

In [13]:
def caption_image(image_path) -> str:
	try:
		caption = blip_caption.caption_image(image_path)
		return caption
	except Exception as e:
		print(f":: Error in caption_image: {e}")
		return "bruh"

In [14]:
def wrapper_for_captions(row: object) -> str:
	bruh = "bruh"
	try:
		caption = row["original_caption"]
		image_path = row["original_image"]
		foo = row.__dict__['_name']  # Fucking silly
		progress.update()

		if caption and len(caption) > 5:
			return caption
		else:
			return blip_caption.caption_image(image_path)
	except Exception as e:
		print(e)
		return bruh

In [15]:
def create_tokens(row: object):
	bruh = "bruh"
	try:
		caption = row["original_caption"]
		progress.update()
		tokens = blip_caption.get_nlk_tokens(caption)
		return tokens
	except Exception as e:
		print(e)
		return bruh

In [16]:
temp = filtered_on_exist.copy()
ddf = dd.from_pandas(temp, npartitions=12)
display(temp)

,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,cars are parked on the side of the road in the...,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,a view of a view of a large green and purple a...,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,a scene of a boat is sitting on the shore of a...,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,starrdust sky with a few stars and a few stars,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,a picture taken from the earth's surface of th...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5
16079,greentext,bgho6WK.jpg,Anon does a little trolling,a screenshote of a text message from a man who...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red dress and a woman in a red dress,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf


In [17]:
progress: tqdm = InnerProgressBar(len(temp), "Captioning-Primary-Images")
display(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: wrapper_for_captions(x), axis=1, meta=('str', object)).compute()

Captioning-Primary-Images: 0it [00:00, ?it/s]

'Total Images: 11514'

global:   0%|          | 0/12 [00:00<?, ?it/s]

Captioning-Primary-Images:  92%|█████████▏| 10556/11514 [00:01<00:00, 19675.51it/s]

In [18]:
display("=== Before Drop ===")
display(temp)
temp.rename(columns={"original_caption": "original_caption_old"}, inplace=True)
temp.rename(columns={"new_column": "original_caption"}, inplace=True)
temp.drop(columns=["original_caption_old"], inplace=True)
display("=== After Drop ===")
display(temp)

'=== Before Drop ==='

,subreddit,file_name,text,original_caption,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,cars are parked on the side of the road in the...,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,a view of a view of a large green and purple a...,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,a scene of a boat is sitting on the shore of a...,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...
...,...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,starrdust sky with a few stars and a few stars,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,a picture taken from the earth's surface of th...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...
16079,greentext,bgho6WK.jpg,Anon does a little trolling,a screenshote of a text message from a man who...,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,a man in a red dress and a woman in a red dress,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress


'=== After Drop ==='

,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress


In [19]:
display(f"Saving to parquet {parquet_process_data_path}")
temp.to_parquet(parquet_process_data_path)
del temp

'Saving to parquet /data/parquet/processed_data.parquet'

In [20]:
display(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")
processed_with_captions = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions)

'Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions'

,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress


In [21]:
temp = processed_with_captions.copy()
ddf = dd.from_pandas(temp, npartitions=6)

In [22]:
progress = InnerProgressBar(len(temp), "Captioning-Tokens-For-Image")
display(f"Total Images: {progress.total}")
temp['new_column'] = ddf.apply(lambda x: create_tokens(x), axis=1, meta=('str', object)).compute()


Captioning-Primary-Images: 100%|██████████| 11514/11514 [00:02<00:00, 4872.12it/s] 


'Total Images: 11514'

global:   0%|          | 0/6 [00:00<?, ?it/s]


Captioning-Tokens-For-Image: 100%|█████████▉| 11513/11514 [00:28<00:00, 331.70it/s]

In [23]:
display("=== Before Drop ===")
display(temp)
temp.rename(columns={"thumbnail_caption": "thumbnail_caption_old"}, inplace=True)
temp.rename(columns={"new_column": "thumbnail_caption"}, inplace=True)
temp.drop(columns=["thumbnail_caption_old"], inplace=True)
display("=== After Drop ===")
display(temp)

'=== Before Drop ==='

,subreddit,file_name,text,thumbnail_path,thumbnail_caption,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,new_column
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the...",True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...,"[(cars, NNS), (are, VBP), (parked, VBN), (on, ..."
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a...",True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...,"[(blonde, NNS), (woman, NN), (with, IN), (hair..."
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin...",True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...,"[(man, NN), (with, IN), (beard, NN), (and, CC)..."
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C...",True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...,"[(view, NN), (of, IN), (large, JJ), (green, JJ..."
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting...",True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...,"[(scene, NN), (of, IN), (boat, NN), (is, VBZ),..."
...,...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st...",True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars,"[(starrdust, NN), (sky, NN), (with, IN), (few,..."
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e...",True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...,"[(picture, NN), (taken, VBN), (from, IN), (the..."
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN...",True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...,"[(screenshote, NN), (of, IN), (text, JJ), (mes..."
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",...",True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress,"[(man, NN), (in, IN), (red, JJ), (dress, NN), ..."


'=== After Drop ==='

,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...,"[(cars, NNS), (are, VBP), (parked, VBN), (on, ..."
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...,"[(blonde, NNS), (woman, NN), (with, IN), (hair..."
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...,"[(man, NN), (with, IN), (beard, NN), (and, CC)..."
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...,"[(view, NN), (of, IN), (large, JJ), (green, JJ..."
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...,"[(scene, NN), (of, IN), (boat, NN), (is, VBZ),..."
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars,"[(starrdust, NN), (sky, NN), (with, IN), (few,..."
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...,"[(picture, NN), (taken, VBN), (from, IN), (the..."
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...,"[(screenshote, NN), (of, IN), (text, JJ), (mes..."
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress,"[(man, NN), (in, IN), (red, JJ), (dress, NN), ..."


In [24]:
display(f"Saving to parquet {parquet_process_data_path} with Updated Thumbnail Captions")
temp.to_parquet(parquet_process_data_path)
del temp

'Saving to parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions'

In [25]:
display(f"Reading from parquet {parquet_process_data_path} with Updated Thumbnail Captions")
processed_with_captions_more = pandas.read_parquet(parquet_process_data_path)
display(processed_with_captions_more)

'Reading from parquet /data/parquet/processed_data.parquet with Updated Thumbnail Captions'

,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...,"[[cars, NNS], [are, VBP], [parked, VBN], [on, ..."
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...,"[[blonde, NNS], [woman, NN], [with, IN], [hair..."
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...,"[[man, NN], [with, IN], [beard, NN], [and, CC]..."
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...,"[[view, NN], [of, IN], [large, JJ], [green, JJ..."
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...,"[[scene, NN], [of, IN], [boat, NN], [is, VBZ],..."
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars,"[[starrdust, NN], [sky, NN], [with, IN], [few,..."
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...,"[[picture, NN], [taken, VBN], [from, IN], [the..."
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...,"[[screenshote, NN], [of, IN], [text, JJ], [mes..."
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress,"[[man, NN], [in, IN], [red, JJ], [dress, NN], ..."


In [26]:
display("Filtering Subreddits with Images By original_caption")
filtered_captions = processed_with_captions_more[
	(processed_with_captions_more["original_caption"] != "bruh") &
	(~processed_with_captions_more["original_caption"].isna() | ~ processed_with_captions_more[
		"original_caption"].isnull())
	]

filtered_captions_display = filtered_captions.groupby("subreddit").size().reset_index(name="count")

display(filtered_captions_display.sort_values("count", ascending=False))
display(f"Total Records {filtered_captions_display['count'].sum()}")

'Filtering Subreddits with Images By original_caption'

,subreddit,count
3,EarthPorn,1883
2,CityPorn,1795
13,memes,1180
16,spaceporn,1101
4,Faces,1028
9,SFWRedheads,984
8,SFWNextDoorGirls,848
12,greentext,713
17,trippinthroughtime,478
15,sfwpetite,288


'Total Records 11514'

In [27]:
display("Filtering Subreddits with Images By thumbnail_caption")
filtered_captions_by_thumbnail = filtered_captions[
	(processed_with_captions_more["thumbnail_caption"] != "bruh") &
	(~processed_with_captions_more["thumbnail_caption"].isna() | ~ processed_with_captions_more[
		"thumbnail_caption"].isnull())
	]
filtered_captions_by_thumbnail_display = filtered_captions_by_thumbnail.groupby("subreddit").size().reset_index(name="count")
display(filtered_captions_by_thumbnail_display.sort_values("count", ascending=False))
display(f"Total Records {filtered_captions_by_thumbnail_display['count'].sum()}")

'Filtering Subreddits with Images By thumbnail_caption'

D:\workspaces\General\venv\lib\site-packages\pandas\core\ops\array_ops.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


,subreddit,count
3,EarthPorn,1883
2,CityPorn,1795
13,memes,1180
16,spaceporn,1101
4,Faces,1028
9,SFWRedheads,984
8,SFWNextDoorGirls,848
12,greentext,713
17,trippinthroughtime,478
15,sfwpetite,288


'Total Records 11514'

In [28]:
display("Updating Cloud Storage With Filtered Captioned Images")
def update_cloud(row):
	try:
		_table_adapter = TableAdapter()
		partition_key = "training"
		row_key = row["id"]
		caption = row["original_caption"]
		updated_caption = row["thumbnail_caption"]
		entity = _table_adapter.get_entity("training", partition_key, row_key)
		entity["caption"] = caption
		entity["updated_caption"] = updated_caption
		_table_adapter.upsert_entity_to_table("training", entity)
		progress.update()
		return "bruh"
	except Exception as e:
		print(e)
		return "bruh"

'Updating Cloud Storage With Filtered Captioned Images'

In [29]:
import json
temp = processed_with_captions_more.copy()
temp['foo'] = temp.apply(lambda x: ",".join([json.dumps("|".join([f for f in item.tolist()])) for item in x['thumbnail_caption'].tolist()]), axis=1)
temp.rename(columns={"thumbnail_caption": "thumbnail_caption_old"}, inplace=True)
temp.rename(columns={"foo": "thumbnail_caption"}, inplace=True)
temp.drop(columns=["thumbnail_caption_old"], inplace=True)
display(temp)

,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,cars are parked on the side of the road in the...,"""cars|NNS"",""are|VBP"",""parked|VBN"",""on|IN"",""the..."
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,blonde woman with blonde hair and tattoos on h...,"""blonde|NNS"",""woman|NN"",""with|IN"",""hair|NN"",""a..."
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a man with a beard and a beard sitting in fron...,"""man|NN"",""with|IN"",""beard|NN"",""and|CC"",""sittin..."
5,spaceporn,7s5aafaqkb9a1.jpg,Northern Lights above Lofoten,D:\data\images\spaceporn\thumbnail\7s5aafaqkb9...,True,D:\data\images\spaceporn\7s5aafaqkb9a1.jpg,True,2c39ce1290fba541abd0b004b09da6b2,1000mjs,a view of a view of a large green and purple a...,"""view|NN"",""of|IN"",""large|JJ"",""green|JJ"",""and|C..."
7,spaceporn,abojw7lqlb9a1.jpg,Viking Lights,D:\data\images\spaceporn\thumbnail\abojw7lqlb9...,True,D:\data\images\spaceporn\abojw7lqlb9a1.jpg,True,0f72de47c69ff50eca5fa3990215f4ac,1000qpd,a scene of a boat is sitting on the shore of a...,"""scene|NN"",""of|IN"",""boat|NN"",""is|VBZ"",""sitting..."
...,...,...,...,...,...,...,...,...,...,...,...
16077,spaceporn,abwhhq0w8b9a1.jpg,Polaris to Cassiopeia on a cloudy night.,D:\data\images\spaceporn\thumbnail\abwhhq0w8b9...,True,D:\data\images\spaceporn\abwhhq0w8b9a1.jpg,True,f5973637fc56360c15818ba0ca1f7ffa,zzz6dp,starrdust sky with a few stars and a few stars,"""starrdust|NN"",""sky|NN"",""with|IN"",""few|JJ"",""st..."
16078,spaceporn,7hzipg1bab9a1.jpg,The hunt for habitable ocean worlds beyond our...,D:\data\images\spaceporn\thumbnail\7hzipg1bab9...,True,D:\data\images\spaceporn\7hzipg1bab9a1.jpg,True,5b22bea7582229c1f9b992176a2ca2c6,zzzcn5,a picture taken from the earth's surface of th...,"""picture|NN"",""taken|VBN"",""from|IN"",""the|DT"",""e..."
16079,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,a screenshote of a text message from a man who...,"""screenshote|NN"",""of|IN"",""text|JJ"",""message|NN..."
16081,trippinthroughtime,arCpzQ0.jpg,He didn't shed light on the topic I guess.,D:\data\images\trippinthroughtime\thumbnail\ar...,True,D:\data\images\trippinthroughtime\arCpzQ0.jpg,True,5007b937974ae333022c0c91b795ca09,zzzlbf,a man in a red dress and a woman in a red dress,"""man|NN"",""in|IN"",""red|JJ"",""dress|NN"",""and|CC"",..."


In [30]:
progress = InnerProgressBar(len(processed_with_captions_more), "upload to cloud")
ddf = dd.from_pandas(temp, npartitions=12)
ddf.apply(lambda x: update_cloud(x), axis=1, meta=('str', object)).compute()
print("Process Complete: Items Updated")

Captioning-Tokens-For-Image: 100%|██████████| 11514/11514 [00:30<00:00, 375.35it/s]


global:   0%|          | 0/12 [00:00<?, ?it/s]

upload to cloud: 100%|██████████| 11514/11514 [03:55<00:00, 11.11it/s]

Process Complete: Items Updated
